In [1]:
# Данные для демонстрации примеров отбора признаков
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification

pd.options.display.float_format = "{:,.2f}".format      # функция форматирования для каждого выводимого числа

X,y = make_classification(n_samples= 10_000, n_features=10, n_redundant=4, random_state=24675)
X = pd.DataFrame(columns=[f"x{i}" for i in range(X.shape[1])], data=X)
X['x10'] = np.random.uniform(low=-0.1, high=0.1, size=len(X))
X['x11'] = 42
X.head()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
0,1.46,-0.35,0.05,-0.79,0.80,0.45,0.87,0.87,0.72,0.89,-0.10,42
1,-0.69,-0.68,0.16,0.25,0.08,0.78,1.06,0.58,-0.14,1.21,0.05,42
2,-0.24,2.00,-0.75,-0.46,-0.47,-1.99,-0.96,2.07,0.05,-1.86,-0.06,42
3,0.18,-0.69,0.28,-0.24,0.30,0.66,0.16,-1.00,-1.59,0.51,-0.10,42
4,-0.11,-1.23,0.39,0.58,0.78,1.30,1.14,-0.30,1.02,1.58,-0.10,42


In [2]:
X.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
count,"10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00","10,000.00"
mean,0.02,0.00,-0.00,-0.00,0.01,-0.00,-0.01,-0.01,0.01,-0.01,-0.00,42.00
std,1.00,0.94,0.33,1.00,0.99,0.98,1.01,1.36,0.99,1.22,0.06,0.00
min,-3.68,-2.35,-1.60,-3.63,-3.58,-3.88,-3.82,-4.76,-3.55,-4.53,-0.10,42.00
25%,-0.66,-0.80,-0.20,-0.68,-0.65,-0.79,-0.71,-1.03,-0.67,-1.00,-0.05,42.00
50%,0.02,-0.12,0.03,0.01,0.01,0.13,-0.05,-0.18,0.03,0.10,0.00,42.00
75%,0.69,0.72,0.22,0.67,0.69,0.87,0.71,1.02,0.69,1.01,0.05,42.00
max,3.78,3.99,0.95,3.69,4.01,2.45,3.83,5.61,4.03,3.89,0.10,42.00


In [46]:
# перед отбором по STD данные нужно масштабировать
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x2 = scaler.fit_transform(X)
X2 = pd.DataFrame( data = x2, columns=X.columns)
X2.describe()

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11
count,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,1.000e+04,10000.0
mean,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.000e+00,0.0
std,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,1.000e+00,0.0
min,-3.704e+00,-2.499e+00,-4.811e+00,-3.636e+00,-3.623e+00,-3.945e+00,-3.788e+00,-3.500e+00,-3.580e+00,-3.705e+00,-1.734e+00,0.0
25%,-6.854e-01,-8.485e-01,-5.912e-01,-6.789e-01,-6.656e-01,-8.002e-01,-6.983e-01,-7.536e-01,-6.820e-01,-8.109e-01,-8.709e-01,0.0
50%,-6.997e-03,-1.323e-01,8.018e-02,8.482e-03,0.000e+00,1.361e-01,-3.901e-02,-1.314e-01,2.055e-02,9.059e-02,0.000e+00,0.0
75%,6.663e-01,7.607e-01,6.532e-01,6.741e-01,6.813e-01,8.908e-01,7.115e-01,7.585e-01,6.904e-01,8.341e-01,8.697e-01,0.0
max,3.758e+00,4.238e+00,2.856e+00,3.695e+00,4.032e+00,2.507e+00,3.812e+00,4.136e+00,4.051e+00,3.192e+00,1.722e+00,0.0


# Отбор признаков (feature selection)

**Зачем?**
- Убирает проблему мультиколлинаераности (актуальна для некоторых моделей) 
- Упрощает модель
    - следовательно упрощает задачу обучения модели. Больше шансов построить более точную модель.
    - модель можно обучить быстрее
    - простые модели менее склонны к переобучению.

**проклятье размерности (curse of dimensionality)** - проблемы возникающие при большом количестве признаков:
- возможны случайные зависимости между признаками (в том числе между целевым и независимыми)
- повышенное потребление памяти 

**Ссылки**
1. https://scikit-learn.org/stable/modules/feature_selection.html


## Критерии отбора
#### 1. На основе изменчивости признаков
Удалять все константные признаки или признаки с маленьким стандартным отклонением.

In [33]:
from sklearn.feature_selection import VarianceThreshold

print(f"shape before filtering: {X2.values.shape}")
var_filter = VarianceThreshold(threshold=0.1)

x3 = var_filter.fit_transform(X2)
print( var_filter.feature_names_in_ )               # исходные признаки
print( var_filter.get_feature_names_out() )         # отфильтрованные признаки
print(f"shape before filtering: {x3.shape}")

shape before filtering: (10000, 11)
['x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x6' 'x7' 'x8' 'x9' 'x10']
['x0' 'x1' 'x2' 'x3' 'x4' 'x5' 'x6' 'x7' 'x8' 'x9']
shape before filtering: (10000, 10)


#### 2. Фильтрационные методы - отбрасывать признаки по одному, независимо друг от друга
- По коэффициенту корреляции Пирсона (линейному к.к.).
    - Выбирать признаки, которые имеют сильную корреляцию с целевым признаком. Например $|r| > 0.2$
    - Отбросить те признаки, для которых есть парный признак с высокой корреляцией.\
    Это поможет избавиться от почти линейной зависимости между признаками, которая вредит некоторым моделям (линейная и логистическая регрессия).
- Другие показатели взаимосвязи признаков (например нелинейные К.К.). См. также пакет `phik`



#### 3. На основе показателей (или параметров) некоторых моделей (Model-based method)
- Коэффициенты перед признаками в линейной или логистической регрессии.\
    Чем больше коэффициент тем более значим признак. *Признаки должны быть приведены к одной и той же шкале*\
- Значимость признаков на основе случайного леса или градиентного бустинга.

Во время обучения дерева ([пример](https://colab.research.google.com/drive/1Bin_h7BPSfnxs4Pea7eibceNMkOEcKmg?usp=sharing)) нужно решать два вопроса: какой признак использовать для разделения узла на ветви, какую границу значения признака выбрать. 
Выбирается тот признак, который даёт больше прироста информации (позволяет создать более эффективное разделение). Для оценки этого показателя используются или коэффициент Джини или показатель информационной энтропии.
Чем чаще признак использовался для разделения, тем важнее признак. Обычно это количество делиться на общее количество деревьев (или узлов во всех деревьях)

В SKlearn есть специальный класс, который способен автоматически отбирать признаки, вне зависимости от того, какая модель предоставляет показатели важности признаков - `SelectFromModel`.


Такое измерение важности признаков, в отличии, например от К.К., позволяет оценить важность отдельного признака, но при этом все признаки учитываются в совокупности.

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

clf = RandomForestClassifier()
clf.fit(X, y)

print("feature_importances")
print( "  ".join( [f"x{i:<4}" for i in range(len(clf.feature_importances_))]) )
print( "  ".join( [f"{x:5.3f}" for x in clf.feature_importances_]) )

# отбирает все признаки, у которых вес (weight или показатель важности), полученный из модели МО, превышает установленное пороговое значение
feature_selector = SelectFromModel( clf, threshold="median", prefit=True)
# threshold = {median, mean, number} - порог отбора признаков
# max_features (default = None) - задаёт максимальное количество признаков, нужно задать threshold=-np.inf если использовать этот параметр
# prefit - используется уже обученная модель?
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html

# преобразуем данные
X_filtred = feature_selector.fit_transform(X, feature_names = X.columns)

print(f"\nРазмерность новых признаков: {X_filtred.shape[1]}")
feature_selector.get_feature_names_out()

feature_importances
x0     x1     x2     x3     x4     x5     x6     x7     x8     x9   
0.017  0.120  0.072  0.018  0.017  0.217  0.156  0.049  0.018  0.316

Размерность новых признаков: 5


array(['x1', 'x2', 'x5', 'x6', 'x9'], dtype=object)




#### 4. Встроенные методы (Embedded methods) - отбор признаков - часть процесса обучения модели
См. регуляризации (L1 или L2) в линейных моделях.

#### 5. (Wrapper methods - ручной или автоматизированный отбор признаков на основе оценки модели на тестовой выборке.

1. Обучить модель. Оценить на тестовой выборке. Удалить признак. Повторить обучение и оценку. Удалить признак окончательно если качество выросло. Повторить для всех признаков или комбинаций признаков.
2. Аналогичный алгоритм, но начинать с одного признака и добавлять признаки.\

```python
from sklearn.feature_selection import SequentialFeatureSelector
bfs = SequentialFeatureSelector(lr, k_features='best', forward = False, n_jobs=-1)
```
3. Исчерпывающий выбор признаков - полный перебор всех комбинаций признаков.
```python
from mlxtend.feature_selection import ExhaustiveFeatureSelector
```
- см. Recursive feature elimination


#### 5. Уменьшение размерности
PCA, UMAP и др. методы

In [1]:
from sklearn.decomposition import PCA
# создаем объект PCA
pca = PCA(n_components=2)
# обучаем модель и преобразуем данные
X_transformed = pca.fit_transform(X)


ModuleNotFoundError: No module named 'sklearn'

# Извлечение признаков (feature extraction)